In [21]:
import torch
import torch.nn as nn
import torch.optim
import torchvision
from torchvision import transforms
from torchvision.datasets import SVHN
from torch.utils.data import random_split, DataLoader

from torchmetrics.classification import Accuracy
import torchmetrics

In [22]:
!pip install torchmetrics

In [23]:
mean= (0.4377, 0.4438, 0.4728)
std= (0.1980, 0.2010, 0.1970)

#Reference for these values:
# Convolutional Neural Networks Applied to House Numbers Digit Classification
# Pierre Sermanet, Soumith Chintala and Yann LeCun

In [24]:
train_transform= transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(0.2, 0.2, 0.2),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

val_transform= transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

test_transform= transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [25]:
dataset= SVHN(root="data", split="train", download=True, transform= train_transform)
test_set= SVHN(root="data", split="test", download=True, transform= test_transform)
len(dataset)

73257

In [26]:
train_length= int(0.8 * len(dataset))

val_length= int(len(dataset)-train_length)

print(train_length, val_length)

58605 14652


In [27]:
train_set, val_set= torch.utils.data.random_split(dataset,[train_length, val_length])

In [28]:
print(len(train_set), len(val_set), len(test_set))

58605 14652 26032


In [29]:
train_loader= DataLoader(train_set, shuffle=True, batch_size= 128)
val_loader= DataLoader(val_set, shuffle= False, batch_size=128)
test_loader= DataLoader(test_set, shuffle=False, batch_size=128)

In [30]:
class CNN(nn.Module):
  def __init__(self):
    super().__init__()

    # Input image is 32x32

    self.stack= nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),  # In output here no. of channels will remain the same

        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),

        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),

        nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.classifier= nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=256*2*2, out_features=512),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(in_features=512, out_features=10)
    )

  def forward(self, x):
    x=self.stack(x)
    x=self.classifier(x)
    return x



In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [32]:
model= CNN().to(device)

In [33]:
loss_function= nn.CrossEntropyLoss()

In [34]:
optimizer= torch.optim.Adam(model.parameters(), lr= 0.001)

scheduler= torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode= "max", factor= 0.5, patience= 3)

In [35]:
def train(dataloader, model, loss_function, optimizer):
  model.train()
  total_loss= 0

  for batch, (image, label) in enumerate(dataloader):
    image= image.to(device)
    label= label.to(device)

    prediction= model(image)
    loss= loss_function(prediction, label)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    total_loss+= loss

  avg_loss= total_loss / len(dataloader)
  print(f"Training Average Loss: {avg_loss:.4f}")

In [36]:
val_accuracy = Accuracy(task="multiclass", num_classes=10).to(device)

def validate(dataloader, model, loss_function):
    model.eval()
    total_loss = 0
    val_accuracy.reset()

    with torch.no_grad():
        for image, label in dataloader:
            image = image.to(device)
            label = label.to(device)

            pred = model(image)
            loss = loss_function(pred, label)
            total_loss += loss

            val_accuracy.update(pred, label)

    avg_loss = total_loss / len(dataloader)
    accuracy = val_accuracy.compute() * 100  # Convert to %

    print(f"Validation Loss: {avg_loss:.4f}, Validation Accuracy: {accuracy:.2f}%")

    return accuracy

In [39]:
test_accuracy = Accuracy(task="multiclass", num_classes=10).to(device)

def test(dataloader, model, loss_function):
    model.eval()
    total_loss = 0
    test_accuracy.reset()   #Reset metric state for each epoch

    with torch.no_grad():
        for image, label in dataloader:
            image = image.to(device)
            label = label.to(device)

            pred = model(image)
            loss = loss_function(pred, label)
            total_loss += loss

            test_accuracy.update(pred, label)

    avg_loss = total_loss / len(dataloader)
    accuracy = test_accuracy.compute() * 100  # Convert to %

    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%")

    return accuracy

In [38]:
epochs = 35
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train(train_loader, model, loss_function, optimizer)
    val_acc = validate(val_loader, model, loss_function)
    scheduler.step(val_acc)


print("-----------------------------------------------------------------")
test(test_loader, model, loss_function)


Epoch 1/35
Training Average Loss: 1.7664
Validation Loss: 0.8497, Validation Accuracy: 71.03%

Epoch 2/35
Training Average Loss: 0.6673
Validation Loss: 0.5161, Validation Accuracy: 83.33%

Epoch 3/35
Training Average Loss: 0.5105
Validation Loss: 0.4500, Validation Accuracy: 85.87%

Epoch 4/35
Training Average Loss: 0.4388
Validation Loss: 0.3977, Validation Accuracy: 87.76%

Epoch 5/35
Training Average Loss: 0.4026
Validation Loss: 0.4021, Validation Accuracy: 87.29%

Epoch 6/35
Training Average Loss: 0.3736
Validation Loss: 0.3556, Validation Accuracy: 89.02%

Epoch 7/35
Training Average Loss: 0.3544
Validation Loss: 0.3529, Validation Accuracy: 89.13%

Epoch 8/35
Training Average Loss: 0.3359
Validation Loss: 0.3214, Validation Accuracy: 90.31%

Epoch 9/35
Training Average Loss: 0.3170
Validation Loss: 0.3195, Validation Accuracy: 90.61%

Epoch 10/35
Training Average Loss: 0.3092
Validation Loss: 0.3319, Validation Accuracy: 89.97%

Epoch 11/35
Training Average Loss: 0.2969
Valida

/usr/local/lib/python3.11/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: The ``compute`` method of metric MulticlassAccuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)


tensor(0., device='cuda:0')